In [9]:
import pandas as pd
train_data = pd.read_csv("labeledTrainData.tsv", header=0, delimiter="\t", quoting=3)

In [10]:
train_data.shape

(25000, 3)

In [11]:
train_data.columns.values

array(['id', 'sentiment', 'review'], dtype=object)

In [13]:
from bs4 import BeautifulSoup

In [15]:
example1 = BeautifulSoup(train_data["review"][0], "html.parser")

In [20]:
import re

In [25]:
letters_only = re.sub("[^a-zA-Z]", " ", example1.get_text())

In [27]:
lower_case = letters_only.lower()

In [29]:
words = lower_case.split()

In [31]:
import nltk
nltk.download()

showing info http://www.nltk.org/nltk_data/


True

In [34]:
from nltk.corpus import stopwords

In [36]:
words = [w for w in words if not w in stopwords.words("english")]

In [53]:
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")
stemmed = [stemmer.stem(w) for w in words]

In [58]:
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()
lemmaed = [wordnet_lemmatizer.lemmatize(w) for w in words]

In [97]:
def review_to_words(raw_review):
    review_text = BeautifulSoup(raw_review).get_text()
    
    letters_only = re.sub("[^a-zA-Z]", " ", review_text)
    
    words = letters_only.lower().split()
    
    stops = set(stopwords.words("english"))
    
    meaningful_words = [w for w in words if not w in stops]
    
    lemmaed_words = [wordnet_lemmatizer.lemmatize(w) for w in meaningful_words]
    
    return(" ".join(lemmaed_words))

In [92]:
clean_train_reviews = []

for i, review in enumerate(train["review"]):
    if (i % 1000 == 0):
        print "processed reviews", i
    clean_train_reviews.append(review_to_words(review))

processed reviews 0
processed reviews 1000
processed reviews 2000
processed reviews 3000
processed reviews 4000
processed reviews 5000
processed reviews 6000
processed reviews 7000
processed reviews 8000
processed reviews 9000
processed reviews 10000
processed reviews 11000
processed reviews 12000
processed reviews 13000
processed reviews 14000
processed reviews 15000
processed reviews 16000
processed reviews 17000
processed reviews 18000
processed reviews 19000
processed reviews 20000
processed reviews 21000
processed reviews 22000
processed reviews 23000
processed reviews 24000


In [95]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(analyzer="word", tokenizer=None, preprocessor=None, stop_words=None, max_features=5000)

print "Creating bag o' words\n"

train_data_features = vectorizer.fit_transform(clean_train_reviews).toarray()

Creating bag o' words



In [105]:
vocab = vectorizer.get_feature_names()
import numpy as np
dist = np.sum(train_data_features, axis=0)

zipped = zip(vocab, dist)

a = np.array(zipped, dtype=[('vocab', 'S10'), ('dist', int)])

sorted = np.sort(a, order=['dist'])

for tag, count in sorted:
  print count, tag

72 anticipati
72 artsy
72 beware
72 clash
72 critique
72 dandy
72 deleted
72 dilemma
72 disappears
72 embrace
72 eternal
72 evidently
72 exterior
72 fontaine
72 glowing
72 guaranteed
72 headache
72 hepburn
72 historic
72 kidman
72 lester
72 lois
72 mayhem
72 mermaid
72 predator
72 renaissanc
72 rome
72 shell
72 soup
72 steady
72 sympathize
72 taboo
72 virtue
72 wax
72 wilder
73 abrupt
73 ace
73 admirable
73 brendan
73 bronson
73 christine
73 commander
73 corps
73 couch
73 darn
73 deserted
73 gambling
73 gillian
73 gina
73 insurance
73 jess
73 kline
73 lush
73 martha
73 mentality
73 morbid
73 muppet
73 net
73 niece
73 out
73 patricia
73 perception
73 preminger
73 psychology
73 puzzle
73 refer
73 rejected
73 respective
73 route
73 severely
73 surround
73 suspension
73 underworld
74 accepts
74 additional
74 beatles
74 bye
74 compliment
74 controlled
74 cruelty
74 diary
74 dont
74 egg
74 glance
74 grudge
74 heading
74 heartbreak
74 hk
74 hopelessly
74 hughes
74 ignorance
74 injury
74 jacke

In [107]:
from sklearn.ensemble import RandomForestClassifier

forest = RandomForestClassifier(n_estimators = 100)

forest = forest.fit(train_data_features, train['sentiment'])

In [108]:
test_data = pd.read_csv("testData.tsv", header=0, delimiter="\t", quoting=3)
clean_test_reviews = []
for i, review in enumerate(test_data["review"]):
    if (i % 1000 == 0):
        print "processed test reviews", i
    clean_test_reviews.append(review_to_words(review))

test_data_features = vectorizer.transform(clean_test_reviews)
test_data_features = test_data_features.toarray()
result = forest.predict(test_data_features)

output = pd.DataFrame(data={"id":test_data["id"], "sentiment":result})
output.to_csv("Bag_of_Words_Model.csv", index=False,quoting=3)

processed test reviews 0
processed test reviews 1000
processed test reviews 2000
processed test reviews 3000
processed test reviews 4000
processed test reviews 5000
processed test reviews 6000
processed test reviews 7000
processed test reviews 8000
processed test reviews 9000
processed test reviews 10000
processed test reviews 11000
processed test reviews 12000
processed test reviews 13000
processed test reviews 14000
processed test reviews 15000
processed test reviews 16000
processed test reviews 17000
processed test reviews 18000
processed test reviews 19000
processed test reviews 20000
processed test reviews 21000
processed test reviews 22000
processed test reviews 23000
processed test reviews 24000
